<a href="https://colab.research.google.com/github/mon-ika349/projects/blob/main/Movierecommendproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.4 MB/s eta 0:00:00


In [ ]:
with open('Untitled14.ipynb', 'r') as f:
    content = f.read()
print(content)




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.222.202.253:8501

  Stopping...



In [ ]:
import streamlit as st
import pandas as pd
import pickle
import ast # Import ast for literal_eval
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

# Download necessary NLTK data (if not already downloaded)
try:
    nltk.data.find('stem/porter')
except LookupError: # Changed to catch LookupError
    nltk.download('porter') # Changed to download 'porter'

# --- Re-create movies.pkl content ---START
movies = pd.read_csv('/content/tmdb_5000_movies.csv') # Corrected path
credits = pd.read_csv('/content/tmdb_5000_credits.csv') # Corrected path
movies = movies.merge(credits, on='title')
movies = movies[['movie_id', 'genres', 'keywords', 'overview', 'title', 'cast', 'crew']]
movies.dropna(inplace=True)

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert3)
movies['crew'] = movies['crew'].apply(fetch_director)
movies['overview'] = movies['overview'].apply(lambda x: x.split())

movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ', '') for i in x])

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

new_df = movies[['movie_id', 'title', 'tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

# Stemming
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

new_df['tags'] = new_df['tags'].apply(stem)

# Vectorization
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

# Cosine Similarity
similarity = cosine_similarity(vectors)

# Save new_df and similarity matrix to pkl files
pickle.dump(new_df, open('movies.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))
# --- Re-create movies.pkl content ---END

# Now, load the new_df DataFrame containing movie titles and the similarity matrix
new_df = pickle.load(open('movies.pkl', 'rb'))
similarity = pickle.load(open('similarity.pkl', 'rb'))

movie_list = new_df['title'].values

st.title('Movie Recommender System')

selected_movie_name = st.selectbox(
    'Select a movie from the dropdown:',
    movie_list
)

def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_movies = []
    for i in movies_list:
        recommended_movies.append(new_df.iloc[i[0]].title)
    return recommended_movies

if st.button('Get Recommendations'):
    recommendations = recommend(selected_movie_name)
    st.write('### Recommended Movies:')
    for i, movie in enumerate(recommendations):
        st.write(f'{i+1}. {movie}')

with open('app.py', 'w') as f:
    f.write('''
import streamlit as st
import pandas as pd
import pickle

# Load the new_df DataFrame and similarity matrix
new_df = pickle.load(open('movies.pkl', 'rb'))
similarity = pickle.load(open('similarity.pkl', 'rb'))

movie_list = new_df['title'].values

st.title('Movie Recommender System')

selected_movie_name = st.selectbox(
    'Select a movie from the dropdown:',
    movie_list
)

def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_movies = []
    for i in movies_list:
        recommended_movies.append(new_df.iloc[i[0]].title)
    return recommended_movies

if st.button('Show Recommendation'):
    recommendations = recommend(selected_movie_name)
    st.subheader(f'Recommendations for {selected_movie_name}:')
    for i, movie in enumerate(recommendations):
        st.write(f'{i+1}. {movie}')
''')

print("app.py updated and necessary data files created.")

[nltk_data] Error loading porter: Package 'porter' not found in index
/tmp/ipython-input-1112418435.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))
/tmp/ipython-input-1112418435.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())
/tmp/ipython-input-1112418435.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

app.py updated and necessary data files created.


You will need to get a `TMDB API Key` from [themoviedb.org](https://www.themoviedb.org/documentation/api). Once you have it, add it to Colab secrets (under the 🔑 icon in the left panel) as `TMDB_API_KEY`. Then, we can set up the API access.

In [ ]:
import requests
from google.colab import userdata

# Get TMDB API key from Colab secrets
TMDB_API_KEY = userdata.get('TMDB_API_KEY')

if TMDB_API_KEY:
    print("TMDB API key configured.")
else:
    print("TMDB_API_KEY not found in Colab secrets. Please add it.")

TMDB API key configured.


Now, let's update the `app.py` file to include functions for fetching movie posters and displaying them in the Streamlit app.

In [33]:
import streamlit as st
import pandas as pd
import pickle
import ast # Import ast for literal_eval
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import requests # Import requests for API calls

# Download necessary NLTK data (if not already downloaded)
try:
    nltk.data.find('stem/porter')
except LookupError: # Changed to catch LookupError
    nltk.download('porter') # Changed to download 'porter'

# --- Re-create movies.pkl content ---START
movies = pd.read_csv('/content/tmdb_5000_movies.csv') # Corrected path
credits = pd.read_csv('/content/tmdb_5000_credits.csv') # Corrected path
movies = movies.merge(credits, on='title')
movies = movies[['movie_id', 'genres', 'keywords', 'overview', 'title', 'cast', 'crew']]
movies.dropna(inplace=True)

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert3)
movies['crew'] = movies['crew'].apply(fetch_director)
movies['overview'] = movies['overview'].apply(lambda x: x.split())

movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ', '') for i in x])

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

new_df = movies[['movie_id', 'title', 'tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

# Stemming
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

new_df['tags'] = new_df['tags'].apply(stem)

# Vectorization
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

# Cosine Similarity
similarity = cosine_similarity(vectors)

# Save new_df and similarity matrix to pkl files
pickle.dump(new_df, open('movies.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))
# --- Re-create movies.pkl content ---END

# Now, load the new_df DataFrame containing movie titles and the similarity matrix
new_df = pickle.load(open('movies.pkl', 'rb'))
similarity = pickle.load(open('similarity.pkl', 'rb'))

movie_list = new_df['title'].values

st.title('Movie Recommender System')

selected_movie_name = st.selectbox(
    'Select a movie from the dropdown:',
    movie_list
)

def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}/images?api_key={}".format(movie_id, TMDB_API_KEY)
    data = requests.get(url)
    data = data.json()
    if 'posters' in data and len(data['posters']) > 0:
        # Check if there's an English poster, otherwise take the first one
        en_posters = [poster for poster in data['posters'] if poster.get('iso_639_1') == 'en']
        if en_posters:
            return "https://image.tmdb.org/t/p/w500/" + en_posters[0]['file_path']
        else:
            return "https://image.tmdb.org/t/p/w500/" + data['posters'][0]['file_path']
    return None # Return None if no poster is found

def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    recommended_movies = []
    recommended_movies_posters = []
    for i in movies_list:
        movie_id = new_df.iloc[i[0]].movie_id
        recommended_movies.append(new_df.iloc[i[0]].title)
        recommended_movies_posters.append(fetch_poster(movie_id))
    return recommended_movies, recommended_movies_posters

if st.button('Get Recommendations'):
    # The TMDB_API_KEY is now expected to be an environment variable
    if not TMDB_API_KEY:
        st.error("TMDB_API_KEY not found. Please ensure it's set as an environment variable.")
    else:
        names, posters = recommend(selected_movie_name)
        num_recommendations = len(names)

        cols = st.columns(num_recommendations)
        for i in range(num_recommendations):
            with cols[i]:
                st.text(names[i])
                if posters[i]:
                    st.image(posters[i])
                else:
                    st.text("No poster available")

with open('app.py', 'w') as f:
    f.write('''
import streamlit as st
import pandas as pd
import pickle
import requests
import os # Import os module

# Load the new_df DataFrame and similarity matrix
new_df = pickle.load(open('movies.pkl', 'rb'))
similarity = pickle.load(open('similarity.pkl', 'rb'))

movie_list = new_df['title'].values

st.title('Movie Recommender System')

selected_movie_name = st.selectbox(
    'Select a movie from the dropdown:',
    movie_list
)

def fetch_poster(movie_id):
    # Retrieve API key from environment variable
    TMDB_API_KEY = os.environ.get('TMDB_API_KEY')
    if not TMDB_API_KEY:
        return None # Return None if API key is not available
    url = "https://api.themoviedb.org/3/movie/{}/images?api_key={}".format(movie_id, TMDB_API_KEY)
    data = requests.get(url)
    data = data.json()
    if 'posters' in data and len(data['posters']) > 0:
        en_posters = [poster for poster in data['posters'] if poster.get('iso_639_1') == 'en']
        if en_posters:
            return "https://image.tmdb.org/t/p/w500/" + en_posters[0]['file_path']
        else:
            return "https://image.tmdb.org/t/p/w500/" + data['posters'][0]['file_path']
    return None

def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_movies = []
    recommended_movies_posters = []
    for i in movies_list:
        movie_id = new_df.iloc[i[0]].movie_id
        recommended_movies.append(new_df.iloc[i[0]].title)
        poster = fetch_poster(movie_id)
        recommended_movies_posters.append(poster)
    return recommended_movies, recommended_movies_posters

if st.button('Show Recommendation'):
    names, posters = recommend(selected_movie_name)
    num_recommendations = len(names)

    # Display recommendations in columns
    cols = st.columns(num_recommendations)
    for i in range(num_recommendations):
        with cols[i]:
            st.text(names[i])
            if posters[i]:
                st.image(posters[i])
            else:
                st.text("No poster available")
''')

print("app.py updated and necessary data files created.")

[nltk_data] Error loading porter: Package 'porter' not found in index
/tmp/ipython-input-2122251174.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))
/tmp/ipython-input-2122251174.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())
/tmp/ipython-input-2122251174.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

app.py updated and necessary data files created.


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 1s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

In [22]:
!streamlit run app.py &>/content/Untitled14.ipynb & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.222.202.253
⠙⠹your url is: https://big-pets-nail.loca.lt


In [27]:
# Stop any existing Streamlit or localtunnel processes
!kill $(lsof -t -i:8501)
!kill $(lsof -t -i:8501) 2>/dev/null # Attempt to kill again, suppressing errors if already stopped
!pkill -f streamlit # Kill any Streamlit processes
!pkill -f localtunnel # Kill any localtunnel processes
print("Stopped any running Streamlit or localtunnel processes on port 8501.")

Stopped any running Streamlit or localtunnel processes on port 8501.


First, let's make sure any previous `localtunnel` or `streamlit` processes are stopped to avoid port conflicts.

Now, let's install `ngrok`.

In [28]:
!pip install -q pyngrok

You will need to get an `ngrok` authentication token from [ngrok.com](https://ngrok.com/signup). Once you have it, add it to Colab secrets (under the 🔑 icon in the left panel) as `NGROK_AUTH_TOKEN`. Then, we can set up `ngrok`.

In [29]:
from pyngrok import ngrok, conf
import os
from google.colab import userdata

# Get ngrok auth token from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

if NGROK_AUTH_TOKEN:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    print("ngrok authtoken configured.")
else:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")

ngrok authtoken configured.


Now, let's start Streamlit in the background and then expose it using `ngrok`.

In [38]:
# Start Streamlit app in the background
# Set TMDB_API_KEY as an environment variable for the streamlit process
import os
os.environ['TMDB_API_KEY'] = userdata.get('TMDB_API_KEY')

!nohup streamlit run app.py > streamlit_output.log 2>&1 &

from pyngrok import ngrok, conf
from google.colab import userdata

# Re-fetch NGROK_AUTH_TOKEN here to ensure it's available in this scope
NGROK_AUTH_TOKEN_CHECK = userdata.get('NGROK_AUTH_TOKEN')
print(f"NGROK_AUTH_TOKEN_CHECK: {NGROK_AUTH_TOKEN_CHECK is not None}")

if NGROK_AUTH_TOKEN_CHECK:
    # Open a ngrok tunnel to the Streamlit port
    public_url = ngrok.connect(8501)
    print(f"ngrok tunnel object: {public_url}")
    if public_url:
        print(f"Streamlit app running at: {public_url}")
    else:
        print("Failed to create ngrok tunnel. public_url is None.")
else:
    print("NGROK_AUTH_TOKEN not found. Cannot create ngrok tunnel.")
    public_url = None # Explicitly set to None if token is missing

NGROK_AUTH_TOKEN_CHECK: True
ngrok tunnel object: NgrokTunnel: "https://appellate-overcensoriously-nella.ngrok-free.dev" -> "http://localhost:8501"
Streamlit app running at: NgrokTunnel: "https://appellate-overcensoriously-nella.ngrok-free.dev" -> "http://localhost:8501"


In [37]:
import requests
from google.colab import userdata

# Re-fetch TMDB API key from Colab secrets
TMDB_API_KEY = userdata.get('TMDB_API_KEY')

def fetch_poster_test(movie_id):
    if not TMDB_API_KEY:
        print("TMDB_API_KEY not found. Please ensure it's set in Colab secrets.")
        return None
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/images?api_key={TMDB_API_KEY}"
    print(f"Fetching URL: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        print("API Response data:")
        # Print only relevant parts to avoid overwhelming output
        if 'posters' in data and len(data['posters']) > 0:
            print(f"  Number of posters: {len(data['posters'])}")
            first_poster = data['posters'][0]
            print(f"  First poster path: {first_poster.get('file_path')}")
            print(f"  First poster language: {first_poster.get('iso_639_1')}")
            # Construct and return the URL for the first English poster or any poster
            en_posters = [p for p in data['posters'] if p.get('iso_639_1') == 'en']
            if en_posters:
                return "https://image.tmdb.org/t/p/w500/" + en_posters[0]['file_path']
            else:
                return "https://image.tmdb.org/t/p/w500/" + data['posters'][0]['file_path']
        else:
            print("  No 'posters' key or no posters found in API response.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching poster: {e}")
        if response.status_code == 401:
            print("  Error 401: Unauthorized. Please check your TMDB_API_KEY.")
        elif response.status_code == 404:
            print(f"  Error 404: Movie ID {movie_id} not found on TMDB.")
        return None

# Test with a known movie ID, e.g., 'Avatar' has movie_id 19995
test_movie_id = 19995
poster_url = fetch_poster_test(test_movie_id)

if poster_url:
    print(f"Successfully fetched poster URL for movie ID {test_movie_id}: {poster_url}")
else:
    print(f"Failed to fetch poster for movie ID {test_movie_id}.")

Fetching URL: https://api.themoviedb.org/3/movie/19995/images?api_key=b02e2f4c0ed9987c97bbdfff78482755
API Response data:
  Number of posters: 314
  First poster path: /gKY6q7SjCkAU6FqvqWybDYgUKIF.jpg
  First poster language: en
Successfully fetched poster URL for movie ID 19995: https://image.tmdb.org/t/p/w500//gKY6q7SjCkAU6FqvqWybDYgUKIF.jpg


In [36]:
# Stop any existing Streamlit or localtunnel processes
!kill $(lsof -t -i:8501)
!kill $(lsof -t -i:8501) 2>/dev/null # Attempt to kill again, suppressing errors if already stopped
!pkill -f streamlit # Kill any Streamlit processes
!pkill -f localtunnel # Kill any localtunnel processes
print("Stopped any running Streamlit or localtunnel processes on port 8501.")

Stopped any running Streamlit or localtunnel processes on port 8501.


In [32]:
with open('streamlit_output.log', 'r') as f:
    log_content = f.read()
print(log_content)




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.222.202.253:8502

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# Task
Update the `app.py` file to include a Streamlit selectbox for movie selection, a 'Get Recommendations' button, and a placeholder for the actual recommendation logic from the user's notebook, along with dummy recommendations for demonstration.

## Update app.py with movie selection and recommendation placeholder

### Subtask:
Modify the `app.py` file to include a Streamlit selectbox for movie selection, a 'Get Recommendations' button, and a placeholder for the actual recommendation logic from the user's notebook, along with dummy recommendations for demonstration.


## Summary:

### Data Analysis Key Findings
*   The `app.py` file is scheduled for an update to incorporate a user interface built with Streamlit.
*   The planned update includes adding a Streamlit selectbox component to allow users to choose a movie title.
*   A 'Get Recommendations' button will be integrated into the application to initiate the recommendation process.
*   The revised `app.py` will feature a placeholder for the actual recommendation logic, initially displaying dummy recommendations to demonstrate the functionality.

### Insights or Next Steps
*   The immediate next step is to implement the described UI components and the basic interaction flow within `app.py`.
*   Following the UI setup, the focus will shift to integrating the actual recommendation engine's logic, replacing the temporary dummy recommendations.
